# CS 8770 Project 1

## Part 1

### Load libs

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.autograd import Function
import torch.optim as optim

### Load data

In [ ]:
from torchvision import datasets
from torchvision.transforms import ToTensor

train_data = datasets.MNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor())

test_data = datasets.MNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor())

In [ ]:
from torch.utils.data import DataLoader

batch_size = 64

train_dl = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_dl = DataLoader(test_data, batch_size=batch_size, shuffle=True)

### Model definitions

In [ ]:
class MLP(nn.Module):
    
    # H: list of hidden layer dims
    # phi: non-linearity to use
    # n_classes: num of classes to pred
    def __init__(self, H, phi=nn.ReLU(), n_classes=10):
        super(MLP, self).__init__()
        self.flatten = nn.Flatten()
        self.layers = nn.Sequential()
        # create hidden layers based off input list H
        H.insert(0,28*28) # input layer
        [self.layers.append(nn.Linear(h,l)).append(phi) for h, l in zip(H,H[1:])] # hidden layers
        self.layers.append(nn.Linear(H[-1],n_classes)) # output layer

    def forward(self, x):
        x = self.flatten(x) # 28x28 -> 1x784
        return self.layers(x)

In [ ]:
# class RBFNet(nn.Module):
    
#     # clusters: precomputed K-means clusters for init
#     # n_classes: num of classes to pred
#     def __init__(self, clusters, n_classes=10):
#         super(RBFNet, self).__init__()
#         self.K = clusters.shape[0] # number of clusters/RBFs
# #         self.nout = 10
#         self.flatten = nn.Flatten()
#         self.sig = nn.Parameter(torch.ones(self.K,dtype=torch.float64))
#         self.mu = nn.Parameter(torch.from_numpy(clusters))
# #         self.w = nn.Parameter(torch.rand(self.K), requires_grad=True)
# #         self.b = nn.Parameter(torch.rand(self.nout), requires_grad=True)
#         self.lin = nn.Linear(self.K, n_classes)

#     def forward(self, x):
#         N = x.shape[0] # number of samples
#         x = self.flatten(x) # 28x28 -> 1x784
#         rbf_out = torch.zeros(N, self.K, dtype=torch.float32)
#         # RBF pass
#         for i in range(N):
#             for j in range(self.K):
#                 top = (x[i,:]-self.mu[j:]).pow(2).sum().sqrt()
#                 rbf_out[i,j] = torch.exp((-0.5)*(top.pow(2) / self.sig[j].pow(2)))
#         # Perceptron
#         # TODO: this should be softmax for multi-class?
# #         y_pred = torch.sigmoid(torch.mv(rbf_out,self.w)+torch.tensor([self.b]*N, dtype-torch.float64))
#         y_pred = self.lin(rbf_out)
#         return y_pred

In [ ]:
# Single RBF Neuron
class RBFNeuron(nn.Module):

    # mu: RBF mu vector
    # sig: RBF sigma
    def __init__(self, mu, sig):
        super(RBFNeuron, self).__init__()
        self.mu = mu.clone().detach()
        self.sig = sig.clone().detach()

    def __call__(self, x):
        top = (x-self.mu).pow(2).sum(1).sqrt() # ||x-mu||
        return torch.exp((-0.5)*(top / self.sig.pow(2)))

# Layer of RBF Neurons
class RBFLayer(nn.Module):

    # nin: input dim
    # nout: output dim
    # mus: list of mean vectors for RBF neurons
    # sigs: list of sigmas for RBF neurons
    def __init__(self, nin, nout, mus, sigs):
        super(RBFLayer, self).__init__()
        self.neurons = nn.ModuleList([RBFNeuron(mus[i],sigs[i]) for i in range(nout)])

    def __call__(self, x):
        return torch.tensor([f(x).numpy() for f in self.neurons], dtype=torch.float32).transpose(0,1)

# Full RBF Network
class RBFNet(nn.Module):

    # clusters: precomputed K-means clusters for init
    # n_classes: num of classes to pred
    def __init__(self, clusters, n_classes=10):
        super(RBFNet, self).__init__()
        self.K = clusters.shape[0] # number of RBFs
        # should probably be passing in mus, sigs...
        self.mus = nn.Parameter(torch.from_numpy(clusters), requires_grad=False)
        self.sigs = nn.Parameter(torch.ones(self.K, dtype=torch.float64, requires_grad=False)*1e-5)
        self.flatten = nn.Flatten()
        self.layers = nn.Sequential(
            RBFLayer(28*28, self.K, self.mus, self.sigs),
            nn.Linear(self.K, 512),
            nn.ReLU(),
            nn.Linear(512, n_classes)
        )

    def forward(self, x):
        x = self.flatten(x) # 28x28 -> 1x784
        return self.layers(x)

In [ ]:
class CNN(nn.Module):
    
    def __init__(self, size, n_classes=10):
        super(CNN, self).__init__()
        self.flatten = nn.Flatten()
        self.extract = nn.Sequential(
            nn.Conv2d(in_channels=size, out_channels=4,
                      kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Dropout(0.2),
            nn.Conv2d(in_channels=4, out_channels=4,
                      kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Dropout(0.2)
        )
        self.decimate = nn.Sequential(
            nn.Linear(7*7*4, 12),
            nn.ReLU(inplace = True),
            nn.Dropout(0.2),
            nn.Linear(12, n_classes)
        )
        
    def forward(self, x):
        x = self.extract(x)
        x = self.flatten(x)
        y_pred = self.decimate(x)
        
        return y_pred

### Train (& validation / test?) loops

In [ ]:
from tqdm.notebook import tqdm # status bar

def train(model, data, loss_fn, optimizer, epochs=5):

    for epoch in range(epochs):

        epoch_loss = []

        for batch, (samples, labels) in enumerate(tqdm(data)):

            # we need to convert these into tensors
            #samples = samples.type('torch.FloatTensor')
            #labels = labels.type('torch.LongTensor')

            # forward pass
            prediction = model(samples)
            loss = loss_fn(prediction, labels)

            # backprop
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # record loss
            epoch_loss.append(loss.item())

        # keep track of loss over our batches
        #epoch_loss = statistics.mean(epoch_loss)
        print(loss)

### Fit models

In [ ]:
loss_fn = nn.CrossEntropyLoss()

In [ ]:
# hyperparams
learning_rate = 1e-3
momentum = 0.3
epochs = 3

### MLP

In [ ]:
mlp_model = MLP([128,128], nn.ReLU(), n_classes=10)

optimizer = optim.Adam(mlp_model.parameters(), lr=learning_rate)

In [ ]:
train(mlp_model, train_dl, loss_fn, optimizer, epochs=epochs)

### RBF

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(1000, init='k-means++', n_init='auto', random_state=0)
kmeans.fit(train_data.data.flatten(1))
#klabels = kmeans.predict(train_data.data.flatten(1))
clusters = kmeans.cluster_centers_.astype(float)

In [ ]:
rbf_model = RBFNet(clusters, n_classes=10)

optimizer = optim.Adam(rbf_model.parameters(), lr=learning_rate)

In [ ]:
train(rbf_model, train_dl, loss_fn, optimizer, epochs=epochs)

### CNN

In [ ]:
cnn_model = CNN(1, n_classes=10)

optimizer = optim.Adam(cnn_model.parameters(), lr=learning_rate)

In [ ]:
train(cnn_model, train_dl, loss_fn, optimizer, epochs=epochs)

### Results

In [ ]:
import seaborn as sn
import pandas as pd

# resub because we are loading our MNIST training data set
test_dl_2 = DataLoader(dataset=test_data, shuffle=True, batch_size=1)

model = mlp_model

confusion_mat = torch.zeros((10,10))
for sample, label in tqdm(test_dl_2):
    
    label = int(label.numpy())

    prediction = model(sample)
    # take the largest output and return integer of which it was (make a classification decision)
    prediction = int(torch.argmax(prediction).numpy())
    
    confusion_mat[label,prediction] += 1
    
df_cm = pd.DataFrame(np.asarray(ConfusionMatrix), index = [i for i in "0123456789"],
                  columns = [i for i in "0123456789"])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True)
plt.show()